In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import timedelta
import requests
from geopy.distance import geodesic
import holidays

# Load the point datasets (social sensing data)
points1 = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/ProxyData/filtered_eBird2024_1km.csv', sep='\t')
points2 = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/ProxyData/filtered_COiNaturalist_1km.csv', sep='\t')
points3 = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/ProxyData/filtered_COFlickr_1km.csv')

# Convert date columns to datetime format and to geodataframes
points1['eventDate'] = pd.to_datetime(points1['eventDate'], errors='coerce')
points2['eventDate'] = pd.to_datetime(points2['eventDate'], errors='coerce')
points3['eventDate'] = pd.to_datetime(points3['eventDate'], errors='coerce')

geometry1 = [Point(xy) for xy in zip(points1['decimalLongitude'], points1['decimalLatitude'])]
points_gdf1 = gpd.GeoDataFrame(points1, geometry=geometry1, crs='EPSG:4326')

geometry2 = [Point(xy) for xy in zip(points2['decimalLongitude'], points2['decimalLatitude'])]
points_gdf2 = gpd.GeoDataFrame(points2, geometry=geometry2, crs='EPSG:4326')

geometry3 = [Point(xy) for xy in zip(points3['decimalLongitude'], points3['decimalLatitude'])]
points_gdf3 = gpd.GeoDataFrame(points3, geometry=geometry3, crs='EPSG:4326')

# Load the trails shapefile containing trails with training data
trails = gpd.read_file('/Users/kyma3609/Downloads/OSMP_Trails/OSMP_Trails.shp')

In [ ]:
# Dictionary to map alternate trail names to dataset names where there are discrepencies
trail_name_mapping = {
    'Boulder Valley Ranch - Sage Trail': 'Boulder Valley - Sage Trail',
}

# List of trails by name that have on-site data from OSMP to train/validate on
selected_trail_names = ['Doudy Draw', 'South Mesa Trail', 'Flatirons Vista', 'Boulder Valley Ranch - Sage Trail', 'Chautauqua Trail', 'Dakota Ridge', 'Eagle Trailhead', 'Foothills South', 'Sanitas Valley Trail', 'South Boulder Creek', 'South Boulder Creek West', 'Coal Seam TH Access', 'Sawhill Ponds', 'East Boulder - Teller Farm', 'East Boulder - White Rocks', 'Lehigh Connector - North', 'East Boulder - Gunbarrel', 'Joder Ranch TH', 'Green Mountain West Ridge', 'South Boulder Creek Marshall', 'Fowler Trail']

# Prepare empty DataFrame to store results for all trails
all_trails_visitation = pd.DataFrame()

# Loop through each trail
for trail_name in selected_trail_names:
    print(f"Processing {trail_name}...")
    dataset_trail_name = trail_name_mapping.get(trail_name, trail_name)
    selected_trail = trails[trails['GISPROD313'] == dataset_trail_name]
    
    if selected_trail.empty:
        print(f"No data for trail: {trail_name}")
        continue
    buffered_trail = selected_trail.buffer(30.48)
    
    if buffered_trail.is_empty.any():
        print(f"Empty buffer for trail: {trail_name}")
        continue
        
# Spatial join to get the social sensing data within the buffer for each dataset
    points_within_buffer_ebird = gpd.sjoin(points_gdf1, gpd.GeoDataFrame(geometry=buffered_trail, crs=buffered_trail.crs), how="inner", predicate="within")
    points_within_buffer_inaturalist = gpd.sjoin(points_gdf2, gpd.GeoDataFrame(geometry=buffered_trail, crs=buffered_trail.crs), how="inner", predicate="within")
    points_within_buffer_flickr = gpd.sjoin(points_gdf3, gpd.GeoDataFrame(geometry=buffered_trail, crs=buffered_trail.crs), how="inner", predicate="within")
    print(f"{trail_name}: {len(points_within_buffer_ebird)} eBird points, {len(points_within_buffer_inaturalist)} iNaturalist points, {len(points_within_buffer_flickr)} Flickr points")
    
    if len(points_within_buffer_ebird) == 0 and len(points_within_buffer_inaturalist) == 0 and len(points_within_buffer_flickr) == 0:
        print(f"No points found within buffer for trail: {trail_name}")
        continue
# Normalize the eventDate to ensure all timestamps are in the same format
    points_within_buffer_ebird['eventDate'] = points_within_buffer_ebird['eventDate'].dt.date
    points_within_buffer_inaturalist['eventDate'] = points_within_buffer_inaturalist['eventDate'].dt.date
    points_within_buffer_flickr['eventDate'] = points_within_buffer_flickr['eventDate'].dt.date
    # Count visitation per day for each dataset
    visitation_ebird = points_within_buffer_ebird.groupby('eventDate').size().reset_index(name='visitation_ebird')
    visitation_inaturalist = points_within_buffer_inaturalist.groupby('eventDate').size().reset_index(name='visitation_inaturalist')
    visitation_flickr = points_within_buffer_flickr.groupby('eventDate').size().reset_index(name='visitation_flickr')
    visitation_ebird['eventDate'] = pd.to_datetime(visitation_ebird['eventDate'])
    visitation_inaturalist['eventDate'] = pd.to_datetime(visitation_inaturalist['eventDate'])
    visitation_flickr['eventDate'] = pd.to_datetime(visitation_flickr['eventDate'])
# Create a DataFrame with all dates in the range
    visitation_start_date = '2020-01-01'
    visitation_end_date = '2024-12-31'
    all_dates = pd.DataFrame({'date': pd.date_range(start=visitation_start_date, end=visitation_end_date)})
# Merge visitation data with all_dates to include zeroes for missing dates
    full_visitation_ebird = all_dates.merge(visitation_ebird, left_on='date', right_on='eventDate', how='left').fillna(0)
    full_visitation_inaturalist = all_dates.merge(visitation_inaturalist, left_on='date', right_on='eventDate', how='left').fillna(0)
    full_visitation_flickr = all_dates.merge(visitation_flickr, left_on='date', right_on='eventDate', how='left').fillna(0)
    full_visitation_ebird.drop(columns=['eventDate'], inplace=True)
    full_visitation_inaturalist.drop(columns=['eventDate'], inplace=True)
    full_visitation_flickr.drop(columns=['eventDate'], inplace=True)
# Combine the visitation data into one DataFrame
    full_visitation = full_visitation_ebird.merge(full_visitation_inaturalist, on='date', how='left')
    full_visitation = full_visitation.merge(full_visitation_flickr, on='date', how='left')
# Calculate the total visitation
    full_visitation['visitation_count'] = full_visitation[['visitation_ebird', 'visitation_inaturalist', 'visitation_flickr']].sum(axis=1).astype(int)
    full_visitation['trail_name'] = trail_name
    all_trails_visitation = pd.concat([all_trails_visitation, full_visitation])
all_trails_visitation.to_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/VisitationModelData_AllTrails.csv', index=False)

print("Processing complete")


In [ ]:
# Load visitation data
visitation_data = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/VisitationModelData_AllTrails.csv')

# Load NOAA weather station data
weather_stations = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/ghcnd_stations_final.csv')

# Function to get the mapped trail name
def get_mapped_trail_name(trail_name, mapping):
    return mapping.get(trail_name, trail_name)

# Select all trails based on the mapped names
selected_trails = trails[trails['GISPROD313'].isin([get_mapped_trail_name(name, trail_name_mapping) for name in selected_trail_names])]
if selected_trails.empty:
    raise ValueError("None of the selected trails were found in the shapefile.")
all_trails_centroid = selected_trails.geometry.centroid.unary_union.centroid
all_trails_centroid_geo = gpd.GeoSeries([all_trails_centroid], crs="EPSG:26913").to_crs(epsg=4326).geometry.iloc[0]
print(f"All trails centroid coordinates (latitude/longitude): {all_trails_centroid_geo.y}, {all_trails_centroid_geo.x}")

# Function to find the closest weather stations
def find_closest_stations(trail_centroid, weather_stations, num_stations=25):
    weather_stations['distance'] = weather_stations.apply(
        lambda row: geodesic((trail_centroid.y, trail_centroid.x), (row['Latitude'], row['Longitude'])).km, axis=1)
    return weather_stations.sort_values('distance').head(num_stations)

# NOAA API token
token = 'AYmKyDYxGCqUWzGOxWcupbuRpfMPWboP'

# Function to get weather data from the NOAA API
def get_weather_data(token, station_ids, start_date, end_date, datatype):
    url = f"https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {'token': token}
    all_data = pd.DataFrame()

    for station_id in station_ids:
        current_start_date = pd.to_datetime(start_date)
        current_end_date = min(current_start_date + timedelta(days=364), pd.to_datetime(end_date))
        station_data = pd.DataFrame()

        while current_start_date <= pd.to_datetime(end_date):
            params = {
                'datasetid': 'GHCND',
                'stationid': f'GHCND:{station_id}',
                'startdate': current_start_date.strftime('%Y-%m-%d'),
                'enddate': current_end_date.strftime('%Y-%m-%d'),
                'datatypeid': datatype,
                'limit': 1000,
                'units': 'metric'
            }

            response = requests.get(url, params=params, headers=headers)
            if response.status_code == 200:
                data = response.json().get('results', [])
                if data:
                    df = pd.DataFrame(data)
                    df['station_id'] = station_id
                    station_data = pd.concat([station_data, df], ignore_index=True)
                else:
                    print(f"No data available for {datatype} at station {station_id} from {current_start_date} to {current_end_date}.")
            else:
                print(f"Error fetching data: {response.status_code} {response.text}")
                break

            current_start_date = current_end_date + timedelta(days=1)
            current_end_date = min(current_start_date + timedelta(days=364), pd.to_datetime(end_date))

        if not station_data.empty:
            all_data = pd.concat([all_data, station_data], ignore_index=True).drop_duplicates(subset=['date'], keep='first')

    return all_data

# Find the closest weather stations and define start/end dates
closest_stations = find_closest_stations(all_trails_centroid_geo, weather_stations)
start_date = visitation_data['date'].min()
end_date = visitation_data['date'].max()

# Pull precipitation and temperature data from closest station for defined timeframe
precipitation_data = get_weather_data(token, closest_stations['Station_ID'], start_date, end_date, 'PRCP')
temperature_max_data = get_weather_data(token, closest_stations['Station_ID'], start_date, end_date, 'TMAX')
visitation_data['date'] = pd.to_datetime(visitation_data['date']).dt.date
precipitation_data['date'] = pd.to_datetime(precipitation_data['date']).dt.date
temperature_max_data['date'] = pd.to_datetime(temperature_max_data['date']).dt.date

# Merge weather data with visitation data
visitation_with_weather = pd.merge(visitation_data, precipitation_data[['date', 'value']], on='date', how='left', suffixes=('', '_precip'))
visitation_with_weather = pd.merge(visitation_with_weather, temperature_max_data[['date', 'value']], on='date', how='left', suffixes=('', '_tmax'))
visitation_with_weather.rename(columns={'value': 'precipitation', 'value_tmax': 'max_temperature'}, inplace=True)
visitation_with_weather.to_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/VisitationModelData_WithWeather.csv', index=False)

print("Weather data has been successfully appended to the visitation dataset.")


In [ ]:
# Load the visitation data with weather included
all_trails_visitation = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/VisitationModelData_WithWeather.csv')
all_trails_visitation['date'] = pd.to_datetime(all_trails_visitation['date'])
visitation_start_date = all_trails_visitation['date'].min()
visitation_end_date = all_trails_visitation['date'].max()

# Generate holidays column for weekly holiday binary variable
us_holidays = holidays.US(years=range(visitation_start_date.year, visitation_end_date.year + 1))
all_trails_visitation['holiday'] = all_trails_visitation['date'].apply(lambda x: 1 if x in us_holidays else 0)

# Add the 'week_start_date' and 'week_of_year' to maintain weekly records
all_trails_visitation['week_start_date'] = all_trails_visitation['date'] - pd.to_timedelta(all_trails_visitation['date'].dt.dayofweek, unit='d')
all_trails_visitation['week_of_year'] = all_trails_visitation['date'].dt.isocalendar().week

# Group by both trail name and week to maintain unique trail data and aggregate weekly
weekly_data = all_trails_visitation.groupby(['trail_name', 'week_start_date', 'week_of_year']).agg({
    'visitation_ebird': 'sum',
    'visitation_inaturalist': 'sum',
    'visitation_flickr': 'sum',
    'visitation_count': 'sum',
    'precipitation': 'sum', 
    'holiday': lambda x: 1 if x.sum() > 0 else 0,
    'max_temperature': 'mean'
}).reset_index()

# Reorder columns for clarity
weekly_data = weekly_data[['trail_name', 'week_start_date', 'week_of_year', 'visitation_ebird',
                           'visitation_inaturalist', 'visitation_flickr', 'visitation_count', 
                           'precipitation', 'holiday', 'max_temperature']]

# Save the final weekly aggregated dataset
weekly_data.to_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/WeeklyVisitationModelData_Trails.csv', index=False)
print("Processing complete. Data saved.")


In [ ]:
from datetime import timedelta

# Load the OSMP visitation data
osmp_visitation_data = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/OSMP_daily_counts_continuous.csv')
osmp_visitation_data['date'] = pd.to_datetime(osmp_visitation_data['date'])
start_date = pd.Timestamp('2019-12-30')
osmp_visitation_data = osmp_visitation_data.dropna(subset=['daily_visit'])
osmp_visitation_data = osmp_visitation_data[osmp_visitation_data['daily_visit'] > 0]
current_date = pd.Timestamp.today().normalize()  
osmp_visitation_data = osmp_visitation_data[osmp_visitation_data['date'] <= current_date]

# Calculate the week start date based on the custom starting date for each entry
osmp_visitation_data['week_start_date'] = osmp_visitation_data['date'].apply(
    lambda x: start_date + timedelta(days=((x - start_date).days // 7) * 7)
)

# Aggregate OSMP data weekly summing the 'daily_count' for each trail
osmp_weekly = osmp_visitation_data.groupby(['trail_name', 'week_start_date']).agg(weekly_count=('daily_visit', 'sum')).reset_index()
osmp_weekly = osmp_weekly[osmp_weekly['weekly_count'] > 0]
osmp_weekly.to_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/OSMP_VisitationWeekly_Cleaned.csv', index=False)

print("OSMP weekly visitation data (cleaned) saved successfully.")

In [ ]:
# Load the model dataset and cleaned OSMP dataset
trail_data = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/WeeklyVisitationModelData_Trails.csv')
osmp_weekly = pd.read_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/OSMP_VisitationWeekly_Cleaned.csv')
trail_data['week_start_date'] = pd.to_datetime(trail_data['week_start_date'])
osmp_weekly['week_start_date'] = pd.to_datetime(osmp_weekly['week_start_date'])

# Uncomment and adjust the below code if there are discrepancies in trail names
# osmp_weekly['trail_name'] = osmp_weekly['trail_name'].replace({
#    'Doudy Draw Trailhead': 'Doudy Draw',
#    'Marshall Mesa Trailhead': 'Marshall Mesa',
#    'Sanitas Valley Trail': 'Sanitas Valley'
# })

# Merge the OSMP weekly data with the trail data
combined_data = pd.merge(trail_data, osmp_weekly, on=['trail_name', 'week_start_date'], how='left')
combined_data = combined_data.dropna(subset=['weekly_count'])
combined_data = combined_data[combined_data['weekly_count'] > 0]
combined_data.to_csv('/Users/kyma3609/Documents/Recreation_Wildfire/MORPHO/CombinedWeeklyVisitationData.csv', index=False)

print("Data successfully merged, cleaned, and saved.")